# HMM GMM

We use 3-state mono-phone HMMs to construct this recognizer. The emission probability of every state is modeled by a GMM. Say we have F mono-phones (F = 39 in our English lexicon), and a G-mixture GMM for each mono-phone state. Thus the GMM-HMM has 3FG mixture components in total. Compared to GMM-UBM, these mixtures are better separated in the phoneme space.

MFCC:  MFCCs  are  extracted  from  16kHz  utterance  with  40 filter-banks distributed between 0 and 8kHz. Static 19-dimensional coefficients plus energy and their delta and delta-delta form a 60-dimensional vector. CMVN is applied per utterance.

Given a transcription, a graph of HMM is composed.

Speaker adaption is the same with Eq. 4 except mixtures here are phonetic dependent.

During the test phase,  the Viterbi-based log-likelihood ratio is expressed as:
sum_t log P(x_t | model_user,qt) - log P(x_t | model_ubm,qt)

HMM: To generate the alignment for the HMM-based modeling,we use MFCCs to train the HMM. 39 mono-phones plus a silencemodel are used,  each of which contains 3 states.  To model thecomplexity of silence, a GMM with 16 mixtures is used for every silence state, while other states are all modeled by 8 Gaussians,resulting 984 Gaussians in total. This HMM is further extended toa triphone system and remains 2142 senones.  The transcriptionsfor  DNN  training  is  generated  by  the  senone  alignment.   OnlyMFCCs are used for HMM training and alignment.

GMM-HMM and i-vector/HMM: The GMM of every state is re-estimated using the HMM alignments and different speaker fea-tures.   The total number of mixtures in our model is 984.   Thedimension of i-vector is again set to 600.  Viterbi and FB align-ments are both investigated

* 39 x 3 states

## Hmm, no dobra, plan działania:

Preprocessing:
wytnij ciszę z dźwięków, voice activity detection 
Wylicz MFCC, 20 stanów + d + dd 

Training:
Dla każdego sentence_id (jest ich 10):
    Utwórz HMMGMM z całego zbioru treningowego sequence_id:
        HMM with 3 states and 8 Gaussian components for each of 39 mono-phones were used for the alignment (resulting in to-tal number of 936 Gaussian components) ?? 
        albo unique(znaki) * 3 stany
        albo znaki * 3 stany trójkątna! (Bakis)
        
Enrollment:
Dla każdego gspeaker_id:
    Dla każdego sentence_id:
        Weź dane (gspeaker_id, sequence_id)
        Naucz HMM taki jak wyżej tylko na nich
        Dokonaj MAP adaptacji
        
Trial:
likelihood HMM_{podany speaker, podane zdanie} - likelihood HMM_{UBM, podane zdanie} ? 

In [1]:
%load_ext autoreload

In [16]:
%autoreload 2

import concurrent.futures as cf
import functools as ft
import itertools as it
import json
import math
import operator as op
import os
import re

from IPython.display import display
from ipywidgets import interact, interact_manual, widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from scipy import interpolate, linalg, misc, optimize, spatial, stats
from sklearn import metrics, mixture, cluster, utils

from paprotka.dataset import reddots
from paprotka.feature import cepstral

In [17]:
%autoreload 0

In [4]:
import warnings
warnings.filterwarnings('once')

# Load

In [5]:
root = reddots.get_root()
load_pcm = ft.partial(reddots.load_pcm, root)
load_mfcc = ft.partial(reddots.load_npy, root, 'wac2_mfcc13_ddd_lifter22_hamming')

In [6]:
all_paths = [os.path.join(os.path.basename(root), file) for root, _, files in os.walk(root + '/pcm') 
                                      for file in files 
                                      if file.endswith('.pcm')]
print(len(all_paths), all_paths[0])

15305 f0002/20150224142650384_f0002_14438.pcm


In [7]:
all_mfcc = {path: load_mfcc(path) for path in all_paths}

In [8]:
script = pd.read_csv(root + '/infos/phones.csv', sep=';')
print(script.loc[script['sentence_id'] == 31])

      sentence_id                  content  \
1609           31  My voice is my password   

                                                 phones  
1609  M AY1 _ V OY1 S _ IH1 Z _ M AY1 _ P AE1 S W ER2 D  


In [9]:
def load_sets(eid, tid=None):
    tid = tid if tid else eid
    
    enrollments = reddots.load_enrollments(root + '/ndx/f_part_{}.trn'.format(eid), 
                                           root + '/ndx/m_part_{}.trn'.format(eid))
    trials = reddots.load_trials(root + '/ndx/f_part_{}.ndx'.format(tid), 
                                 root + '/ndx/m_part_{}.ndx'.format(tid))
    
    enrollments = enrollments.merge(script, how='left', on='sentence_id')
    trials = trials.merge(script, how='left', left_on='expected_sentence_id', right_on='sentence_id')
    del trials['sentence_id']
    trials.rename(columns={'content': 'expected_content', 'phones': 'expected_phones'}, inplace=True)
    trials = trials.merge(script, how='left', left_on='trial_sentence_id', right_on='sentence_id')
    del trials['sentence_id']
    trials.rename(columns={'content': 'trial_content', 'phones': 'trial_phones'}, inplace=True)
    
    return enrollments, trials

In [10]:
enrollments_1, trials_1 = load_sets('01')
print('Enrollments', enrollments_1.dtypes, sep='\n')
print('Trials', trials_1.dtypes, sep='\n')

Enrollments
is_male                  bool
pcm_path               object
sentence_id             int16
speaker_id              int16
timestamp      datetime64[ns]
content                object
phones                 object
dtype: object
Trials
correct_sentence                  bool
expected_is_male                  bool
expected_sentence_id             int16
expected_speaker_id              int16
pcm_path                        object
target_person                     bool
trial_is_male                     bool
trial_sentence_id                int16
trial_speaker_id                 int16
trial_timestamp         datetime64[ns]
expected_content                object
expected_phones                 object
trial_content                   object
trial_phones                    object
dtype: object


In [11]:
enrollments_2, trials_2 = load_sets('02')
enrollments_3, trials_3 = load_sets('03')
enrollments_4_td, trials_4_td = load_sets('04_td', '04')

In [12]:
trialed_paths = set(path for trials in (trials_1, trials_2, trials_3, trials_4_td) for path in trials.pcm_path)
untrialed_paths = [path for path in all_paths if path not in trialed_paths]
print(len(trialed_paths), len(untrialed_paths))

10857 4448


In [13]:
shit_lengths = np.array([mfcc.shape[0] for mfcc in all_mfcc.values()])
print(shit_lengths.min(), shit_lengths.mean(), shit_lengths.max(), shit_lengths)

shit_lengths_sorted = np.sort(shit_lengths)

print(shit_lengths_sorted[:40])
print(shit_lengths_sorted[-40:])

1 316.151323097 1219 [289 398 300 ..., 261 193 370]
[ 1  1  1  7  8  9  9  9 11 12 12 12 12 13 23 27 60 60 78 79 80 85 86 87 87
 87 89 89 89 89 89 89 89 89 91 91 92 92 93 93]
[1047 1047 1057 1058 1059 1059 1063 1065 1065 1067 1067 1067 1069 1069 1071
 1076 1078 1078 1078 1082 1086 1088 1088 1090 1181 1181 1181 1181 1181 1181
 1185 1185 1200 1200 1202 1202 1219 1219 1219 1219]


In [14]:
trialed_paths = [path for path in trialed_paths if all_mfcc[path].shape[0] >= 85]
untrialed_paths = [path for path in untrialed_paths if all_mfcc[path].shape[0] >= 85]

In [15]:
# memory blew up
# enrollments_1.mfcc = enrollments_1.apply(lambda row: load_mfcc(row.pcm_path), axis=1)
# trials_1.mfcc = trials_1.apply(lambda row: load_mfcc(row.pcm_path), axis=1)
# enrollments_4_td.mfcc = enrollments_4_td.apply(lambda row: load_mfcc(row.pcm_path), axis=1)

In [19]:
# got_this_back = pg.GeneralMixtureModel(bg_model.distributions, np.exp(bg_model.weights) * bg_model.weights.sum())
# got_this_back = pg.MultivariateGaussianDistribution(means=bg_model.distributions[0].mu, covariance=bg_model.distributions[0].cov)

def with_opened_file(mode='r'):
    def decorator(fun):
        @ft.wraps(fun)
        def wrapped(path, *args, **kwargs):
            with open(path, mode=mode) as opened:
                return fun(opened, *args, **kwargs)
        return wrapped
    return decorator

def write_model(path, model):
    with open(path, 'wb') as opened:
        pickle.dump(model, opened)
        
def read_model(path):
    with open(path, 'rb') as opened:
        return pickle.load(opened)
    
def write_triples(path, triples):
    for phone, triple in triples.items():
        for i, model in enumerate(triple):
            c = chr(ord('a') + i)
            write_model(path + '/{}-{}'.format(phone, c), model)
            
def read_triples(path, phones):
    triples = {}
    for phone in phones:
        triple = tuple(read_model(path + '/{}-{}'.format(phone, c)) 
                       for c in ('a', 'b', 'c'))
    return triples

# Train

In [20]:
example = enrollments_1[enrollments_1.sentence_id == 31]
example_features = [all_mfcc[path] for path in example['pcm_path']]
example_lengths = [mfcc.shape[0] for mfcc in example_features] 
example_features_stack = np.vstack(example_features)

print(len(example_features), example_features_stack.shape)

114 (37993, 39)


In [32]:
example

,is_male,pcm_path,sentence_id,speaker_id,timestamp,content,phones
0,False,f0002/20150216173043137_f0002_31.pcm,31,2,2015-02-16 17:30:43.137000,My voice is my password,M AY1 _ V OY1 S _ IH1 Z _ M AY1 _ P AE1 S W ER2 D
1,False,f0002/20150216173047224_f0002_31.pcm,31,2,2015-02-16 17:30:47.224000,My voice is my password,M AY1 _ V OY1 S _ IH1 Z _ M AY1 _ P AE1 S W ER2 D
2,False,f0002/20150216173052544_f0002_31.pcm,31,2,2015-02-16 17:30:52.544000,My voice is my password,M AY1 _ V OY1 S _ IH1 Z _ M AY1 _ P AE1 S W ER2 D
30,False,f0004/20150303030806097_f0004_31.pcm,31,4,2015-03-03 03:08:06.097000,My voice is my password,M AY1 _ V OY1 S _ IH1 Z _ M AY1 _ P AE1 S W ER2 D
31,False,f0004/20150303030810511_f0004_31.pcm,31,4,2015-03-03 03:08:10.511000,My voice is my password,M AY1 _ V OY1 S _ IH1 Z _ M AY1 _ P AE1 S W ER2 D
32,False,f0004/20150303030815550_f0004_31.pcm,31,4,2015-03-03 03:08:15.550000,My voice is my password,M AY1 _ V OY1 S _ IH1 Z _ M AY1 _ P AE1 S W ER2 D
60,False,f0005/20150329211618801_f0005_31.pcm,31,5,2015-03-29 21:16:18.801000,My voice is my password,M AY1 _ V OY1 S _ IH1 Z _ M AY1 _ P AE1 S W ER2 D
61,False,f0005/20150329211619628_f0005_31.pcm,31,5,2015-03-29 21:16:19.628000,My voice is my password,M AY1 _ V OY1 S _ IH1 Z _ M AY1 _ P AE1 S W ER2 D
62,False,f0005/20150329211620392_f0005_31.pcm,31,5,2015-03-29 21:16:20.392000,My voice is my password,M AY1 _ V OY1 S _ IH1 Z _ M AY1 _ P AE1 S W ER2 D
90,False,f0006/20150324145723981_f0006_31.pcm,31,6,2015-03-24 14:57:23.981000,My voice is my password,M AY1 _ V OY1 S _ IH1 Z _ M AY1 _ P AE1 S W ER2 D


In [21]:
untrialed_features = [all_mfcc[path] for path in untrialed_paths]
untrialed_features_stack = np.vstack(untrialed_features)

print(len(untrialed_features), untrialed_features_stack.shape)

4445 (1442782, 39)


## Make 8 and 16 mixtures all sentence models

In [22]:
# bg_model = mixture.GaussianMixture(
#     n_components=8, max_iter=10000, covariance_type='diag', 
#     random_state=13, verbose=1
# )
# bg_model.fit(untrialed_features_stack)

Initialization 0
  Iteration 0
  Iteration 10
  Iteration 20
  Iteration 30
Initialization converged: True


GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=10000,
        means_init=None, n_components=8, n_init=1, precisions_init=None,
        random_state=13, reg_covar=1e-06, tol=0.001, verbose=1,
        verbose_interval=10, warm_start=False, weights_init=None)

In [23]:
# silence_model = mixture.GaussianMixture(
#     n_components=16, max_iter=10000, covariance_type='diag', 
#     random_state=13, verbose=1
# )
# silence_model.fit(untrialed_features_stack)

Initialization 0
  Iteration 0
  Iteration 10
  Iteration 20
  Iteration 30
  Iteration 40
  Iteration 50
  Iteration 60
Initialization converged: True


GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=10000,
        means_init=None, n_components=16, n_init=1, precisions_init=None,
        random_state=13, reg_covar=1e-06, tol=0.001, verbose=1,
        verbose_interval=10, warm_start=False, weights_init=None)

In [24]:
# write_model(root + '/models/hmm/bg_model.pickle', bg_model)
# write_model(root + '/models/hmm/silence_model.pickle', silence_model)

In [21]:
bg_model = read_model(root + '/models/hmm/bg_model.pickle')
silence_model = read_model(root + '/models/hmm/silence_model.pickle')

## Clean sentences, build triples

In [30]:
def wrap_with_silence(phones):
    return ['_'] + phones + ['_']

def remove_stress_combinations(phones):
    def remove_single_phone(phone):
        if phone[-1].isdigit():
            return phone[:-1]
        else:
            return phone
        
    return [remove_single_phone(phone) for phone in phones]

clean_phones = lambda phones: wrap_with_silence(remove_stress_combinations(phones))

In [26]:
UNIQUE_PHONES = [
    'AA', 'AE', 'AH', 'AO', 'AW', 'AY', 'B', 'CH', 'D', 'DH', 
    'EH', 'ER', 'EY', 'F', 'G', 'HH', 'IH', 'IY', 'JH', 'K', 
    'L', 'M', 'N', 'NG', 'OW', 'OY', 'P', 'R', 'S', 'SH', 'T', 
    'TH', 'UH', 'UW', 'V', 'W', 'Y', 'Z', 'ZH'
]

In [29]:
def load_ubm_copy():
    return read_model(root + '/models/hmm/bg_model.pickle')

def make_phone_triple():
    return tuple(load_ubm_copy() for _ in range(3))

phone_triples = {
    phone: make_phone_triple() for phone in UNIQUE_PHONES
}

In [ ]:
def most_likely_hidden_viterbi(hidden_weights, visible_weights, initial_hidden, observations):
    joint_probs = np.zeros((observations.size, initial_hidden.size), dtype=np.float64)
    max_probs_ix = np.zeros((observations.size - 1, initial_hidden.size), dtype=np.int32)
    
    for i, observation in enumerate(observations):
        if i == 0:
            joint_probs[0] = initial_hidden * visible_weights[:, observation]    
        else:
            next_state_probs = joint_probs[i - 1, np.newaxis] * hidden_weights
            next_state_argmax = next_state_probs.argmax(axis=1)
            next_state_max = next_state_probs.max(axis=1)
            
            joint_probs[i] = next_state_max * visible_weights[:, observation]
            max_probs_ix[i - 1] = next_state_argmax
    
    most_likely_path = np.zeros(observations.size, dtype=np.int32)
    most_likely_path[-1] = joint_probs[-1].argmax()
    for i in range(observations.size - 1, 0, -1):
        most_likely_path[i - 1] = max_probs_ix[i - 1, most_likely_path[i]]
    
    return most_likely_path, joint_probs, max_probs_ix

In [ ]:
def viterbi(hidden_weights, visible_weights, initial_hidden, features):
    joint_probs = np.zeros((observations.size, initial_hidden.size), dtype=np.float64)
    max_probs_ix = np.zeros((observations.size - 1, initial_hidden.size), dtype=np.int32)
    
    for i, observation in enumerate(observations):
        if i == 0:
            joint_probs[0] = initial_hidden * visible_weights[:, observation]    
        else:
            next_state_probs = joint_probs[i - 1, np.newaxis] * hidden_weights
            next_state_argmax = next_state_probs.argmax(axis=1)
            next_state_max = next_state_probs.max(axis=1)
            
            joint_probs[i] = next_state_max * visible_weights[:, observation]
            max_probs_ix[i - 1] = next_state_argmax
    
    most_likely_path = np.zeros(observations.size, dtype=np.int32)
    most_likely_path[-1] = joint_probs[-1].argmax()
    for i in range(observations.size - 1, 0, -1):
        most_likely_path[i - 1] = max_probs_ix[i - 1, most_likely_path[i]]
    
    return most_likely_path, joint_probs, max_probs_ix

def align_frames(phone_triples, silence_model, features, phones):
    frame_n, _ = features.shape
    phone_n = len(phones)
    frame_mean_width = frame_n / phone_n
    
    
# align_frames(phone_triples, silence_model)

In [27]:
def make_sentence_hmm(phones_cell, phone_triples, silence_model):
    phones = phones_cell.split(' ')
    cleaned_phones = clean_phones(phones)
    hmm = pg.HiddenMarkovModel('-'.join(cleaned_phones))
    
    states = []
    for i, phone in enumerate(cleaned_phones):
        if phone == '_':
            phone_state = pg.State(silence_model.copy(), name='{}-SILENCE'.format(i))
            states.append(phone_state)
        else:
            phone_triple = phone_triples[phone]
            phone_states = (
                pg.State(phone_triple[0], name='{:02}a-{}'.format(i, phone)),
                pg.State(phone_triple[1], name='{:02}b-{}'.format(i, phone)),
                pg.State(phone_triple[2], name='{:02}c-{}'.format(i, phone))
            )
            states.extend(phone_states)
    hmm.add_states(states)
    
    hmm.add_transition(hmm.start, states[0], 1.0)
    for state in states:
        hmm.add_transition(state, state, 0.9)
    for prev, then in zip(states, states[1:]):
        hmm.add_transition(prev, then, 0.1)
    hmm.add_transition(states[-1], hmm.end, 0.1)
    hmm.bake()
    
    return hmm

# make_sentence_hmm(
#     script[script.sentence_id == 31].phones.iloc[0], 
#     phone_distribution_triples, silence_model
# );

In [29]:
recording_regex = re.compile(r'([mf])(\d+)/(\d+)_[mf]\d+_(\d+)')
def parse_recording(value):
    match = recording_regex.match(value)
    if match:
        return match.groups()

unique_sentence_ids = np.sort(np.unique([int(parse_recording(path)[3]) for path in untrialed_paths]))
unique_sentence_ids_scripted = [sentence_id for sentence_id in unique_sentence_ids
                                if (script.sentence_id == sentence_id).any()
                                if isinstance(script[script.sentence_id == sentence_id].phones.iloc[0], str)]
print(len(unique_sentence_ids), len(unique_sentence_ids_scripted))

811 769


In [30]:
sentence_hmms = {
    sentence_id: make_sentence_hmm(
        script[script.sentence_id == sentence_id].phones.iloc[0], phone_distribution_triples, silence_model
    ) for sentence_id in unique_sentence_ids_scripted
}

In [31]:
print(len(sentence_hmms))

769


In [32]:
sentence_sequencess = {
    sentence_id: [all_mfcc[path] for path in untrialed_paths 
                                 if int(parse_recording(path)[3]) == sentence_id
                                 if all_mfcc[path].shape[0] >= len(script[script.sentence_id == sentence_id].phones.iloc[0].split(' ') * 3)]
    for sentence_id in unique_sentence_ids_scripted
}

## Train HMMs

In [33]:
def update_triples(phone_triples):
    empty = 0
    lost = 0
    for phone_triple in phone_triples.values():
        for gmm in phone_triple:
            if gmm.summaries.sum() == 0:
                empty += 1
                continue
            try:
                gmm.from_summaries()
            except Exception:
                gmm.clear_summaries()
                lost += 1
    return empty, lost

def train_epoch(sentence_sequencess, unique_sentence_ids, sentence_hmms, phone_triples):
    logps = []
    
    for sentence_id in unique_sentence_ids:
        print(sentence_id, end=' ')
        
        sentence_hmm = sentence_hmms[sentence_id]
        sentence_sequences = sentence_sequencess[sentence_id]
        weights = np.ones(len(sentence_sequences))
        
        logp = sentence_hmm.summarize(sequences=sentence_sequences, weights=weights, check_input=False)
                                      # algorithm='viterbi', 
        logps.append(logp)
    
    update_triples(phone_triples)
            
    return logps

In [34]:
write_triples(root + '/models/triples-init', phone_distribution_triples)

FileNotFoundError: [Errno 2] No such file or directory: '../input/reddots_r2015q4_v1/models/triples-init/AW-a'

In [ ]:
enrollments_4_td.phones[~enrollments_4_td.phones.isna()].apply(lambda row: len(row.split(' '))).describe()

In [ ]:
for i in range(10):
    for j in range(10):
        logps = train_epoch(sentence_sequencess, unique_sentence_ids_scripted, 
                            sentence_hmms, phone_distribution_triples)
        print('{}-{} logps'.format(i, j), sum(logps))
    path = root + '/models/triples-{}'.format(i)
    os.mkdir(path)
    write_triples(path, phone_distribution_triples)

In [160]:
update_triples(phone_distribution_triples)

(0, 0)

In [243]:
write_triples(root + '/models/triples', phone_distribution_triples)

In [21]:
phone_distribution_triples = read_triples(root + '/models/triples-1', UNIQUE_PHONES)

OSError: Cannot read model AW-a

In [105]:
print(example_sentence_hmm.states[3].distribution.distributions[0].mu)
print(example_sentence_hmm.states[10].distribution.distributions[0].mu)
print(example_sentence_hmm.states[10].name)

[  1.41467275e+01  -7.35344053e+00  -5.56301231e+00  -4.84370700e+00
  -3.26590152e+00  -4.67713110e+00  -2.93247692e+00  -6.21254481e+00
  -4.60166262e+00  -4.32767984e+00  -1.62883341e+00  -2.50476327e+00
  -8.24130769e+00   4.71618683e-01   4.57870227e-01  -1.32589895e+00
  -9.23610116e-01  -9.11397488e-01  -4.48633292e-02   6.83980317e-01
  -3.91819699e-01  -9.57840195e-02  -5.51740595e-01  -4.66018632e-01
  -3.27380013e-01   2.68423365e-03   6.40123001e-03   1.34323619e-01
   6.15838646e-03  -4.11852850e-02   7.89280213e-02  -2.26739330e-02
   5.75374672e-02   6.40953838e-03  -3.25918412e-02   1.42842274e-01
  -1.47236299e-01  -6.17011349e-02  -1.74521876e-02]
[  1.43344790e+01   1.12123728e-01  -4.61673784e+00  -4.37537285e+00
  -3.35688364e+00  -5.19159258e+00  -2.46247097e+00  -5.27950688e+00
  -6.93267329e+00   8.92074965e-01  -1.50063469e+00  -3.92398961e+00
  -1.10409868e+01   2.92374934e-01   1.52636161e+00  -1.38141933e+00
  -6.50400994e-01  -7.84072233e-01   1.30328483e-0

In [71]:
for i in (0, 50, 100):
    predictions = example_sentence_hmm.predict(example_features[i], algorithm='viterbi')
    print(i)
    print('\t'.join(example_sentence_hmm.states[prediction].name for prediction in predictions))

0
_-M-AY-_-V-OY-S-_-IH-Z-_-M-AY-_-P-AE-S-W-ER-D-_-start	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	1-M-beg	1-M-mid	1-M-end	2-AY-beg	2-AY-mid	2-AY-end	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-S

In [76]:
example_sentence_hmm.fit(example_features, algorithm='viterbi', n_jobs=12, verbose=True)

TypeError: Cannot cast ufunc subtract output from dtype('complex128') to dtype('float64') with casting rule 'same_kind'

Exception ignored in: 'pomegranate.hmm.HiddenMarkovModel._from_summaries'
Traceback (most recent call last):
  File "pomegranate/bayes.pyx", line 710, in pomegranate.bayes.BayesModel.from_summaries
  File "pomegranate/distributions.pyx", line 2259, in pomegranate.distributions.MultivariateGaussianDistribution.from_summaries
TypeError: Cannot cast ufunc subtract output from dtype('complex128') to dtype('float64') with casting rule 'same_kind'


TypeError: Cannot cast ufunc subtract output from dtype('complex128') to dtype('float64') with casting rule 'same_kind'

Exception ignored in: 'pomegranate.hmm.HiddenMarkovModel._from_summaries'
Traceback (most recent call last):
  File "pomegranate/bayes.pyx", line 710, in pomegranate.bayes.BayesModel.from_summaries
  File "pomegranate/distributions.pyx", line 2259, in pomegranate.distributions.MultivariateGaussianDistribution.from_summaries
TypeError: Cannot cast ufunc subtract output from dtype('complex128') to dtype('float64') with casting rule 'same_kind'


[1] Improvement: 4.373646683990955	Time (s): 522.3


...

[61] Improvement: 2.1627172827720642e-05	Time (s): 537.9


TypeError: Cannot cast ufunc subtract output from dtype('complex128') to dtype('float64') with casting rule 'same_kind'

Exception ignored in: 'pomegranate.hmm.HiddenMarkovModel._from_summaries'
Traceback (most recent call last):
  File "pomegranate/bayes.pyx", line 710, in pomegranate.bayes.BayesModel.from_summaries
  File "pomegranate/distributions.pyx", line 2259, in pomegranate.distributions.MultivariateGaussianDistribution.from_summaries
TypeError: Cannot cast ufunc subtract output from dtype('complex128') to dtype('float64') with casting rule 'same_kind'


[62] Improvement: -1.4025717973709106e-06	Time (s): 544.4
Total Training Improvement: 2551.5645321204793
Total Training Time (s): 33826.2032


2551.5645321204793

## Enroll

In [54]:
def make_speaker_classifier(inspeaker_classifier):
    n_components = 3 * len(phones)
    initial, transitions = make_matrix_beads(n_components)

    classifier = hmm.GMMHMM(
        n_components=n_components, n_mix=8, covariance_type='diag', init_params='mcw', params='tmcw'
    )

    classifier.startprob_ = initial
    classifier.transmat_ = transitions

GMMHMM(algorithm='viterbi', covariance_type='diag', covars_prior=0.01,
    init_params='stmcw', n_components=39, n_iter=10, n_mix=8,
    params='stmcw', random_state=None, startprob_prior=1.0, tol=0.01,
    transmat_prior=1.0, verbose=False)

In [ ]:
def perform_enrollments(classifier, enrollments):
    labels = enrollments[['is_male', 'speaker_id', 'sentence_id']].values
    features = [load_mfcc(path) for path in enrollments_1['pcm_path']]
    classifier.fit(features, labels)
    

## Trial

In [ ]:
def save_results(label, results):
    path = os.path.join(root, 'result', label)
    with open(path) as opened:
        pickle.dump(results, opened)
        
def load_results(label):
    path = os.path.join(root, 'result', label)
    with open(path) as opened:
        return pickle.load(opened)
    
def perform_trial(classifier, path):
    features = load_mfcc(path)
    return classifier.predict_single_proba(features)

def perform_trials(classifier, trials):
    paths = trials['pcm_path'].unique()
    results = {}
    for path in paths:
        results[path] = perform_trial(classifier, path)
    return results